In [1]:
import os
import glob
from bs4 import BeautifulSoup
import sys 
from PIL import Image
import cv2
import numpy as np
import time
import torch
import torchvision
from torch.utils.data import Dataset
from torchvision import transforms
import os
import random
import utils
import wandb
import joblib
import pandas as pd
import matplotlib.pyplot as plt

### Load Classifer

In [ ]:
classifier = joblib.load("inshore_offshore_clf_model.pkl")

In [ ]:
def predict_in_off(classifier, target):
    # targets == filepath to the downloaded images


    fp = target

    # read in images
    fnames = os.listdir(fp)
    train = []
    for file in fnames:
        train.append(plt.imread(fp + '/{}'.format(file)))
    
    # need to optimize inputs, creating a dataframe has a lot of overhead
    img_df = pd.DataFrame(columns = ['50th', '80th', '90th', '30th'])
    
    for img in train:
        img_vals = np.copy(img)
        img_50 = np.percentile(img_vals,50)
        img_80 = np.percentile(img_vals,80)
        img_90 = np.percentile(img_vals,90)
        img_30 = np.percentile(img_vals,30)

        img_df.append({'50th': img_50, '80th': img_80, '90th': img_90, '30th': img_30}, ignore_index=True)

    return classifier.predict(img_df)

### Load RetinaNet

In [ ]:
retina = torchvision.models.detection.retinanet_resnet50_fpn(num_classes=2)
retina.load_state_dict(torch.load("models/retTest3.pt"))
retina.eval()
#model.to(device)

### Load Faster R-CNN

In [ ]:
faster = torchvision.models.detection.fasterrcnn_resnet50_fpn()
num_classes = 2  # 1 class (ship) + background
# get number of input features for the classifier
in_features = faster.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
faster.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)
faster.load_state_dict(torch.load("models/faster300ep.pt"))
faster.eval()
#model.to(device)

In [ ]:
def num_ship_in_image(type,image):
    # use